In [13]:
import openstudio as ost
from pathlib import Path
import os
import shutil

In [10]:
def load_osm(path_str):
    translator = ost.osversion.VersionTranslator
    path = ost.path(path_str)
    model = translator.loadModel(path)
    if model.empty:
        print(f"Input: {path_str} is not valid. Please check")
    else:
        model = model.get
    return model

In [12]:
def create_single_model(building_type, vintage, climate_zone, osm_directory):
    model = ost.model.model
    epw_file = "Not Applicable"
    prototype_creator = 

In [ ]:
def create_workflows(building_types, vintages, climate_zones, root_directory, epws_path, hash_climate_epw, measures_dir = None, n_runs = 5, efficiency_level = 2):
    Path(root_directory).mkdir(parents = True, exist_ok = True)
    hash_eff_level = {
        '1' : "Low",
        '2' : "Standard",
        '3' : "High"
    }
    out_osw_dir = os.path.join(root_directory, "3-OSWs", f"Efficiency_Level_{hash_eff_level[efficiency_level]}")
    v_osw_paths = []
    for building_type in building_types:
        for climate_zone in climate_zones:
            sub_epws_path = os.path.join(epws_path, hash_climate_epw[climate_zone])
            for vintage in vintages:
                model_name = building_type + "_" + vintage + "_" + climate_zone.split('-')[-1]
                seed_model_folder = os.path.join(root_directory, "1-Seeds", model_name)
                new_model_folder = os.path.join(root_directory, "2-Processed_Models", f"Efficiency_Level_{hash_eff_level[efficiency_level]}", model_name)
                old_osm_path = os.path.join(seed_model_folder, "SR1/in.osm")
                old_epw_path = os.path.join(seed_model_folder, "SR1/in.epw")
                new_osm_path = os.path.join(new_model_folder, f"{model_name}.osm")
                new_epw_path = os.path.join(new_model_folder, f"{model_name}.epw")
                # Creating a raw building model
                create_single_model(building_type, vintage, climate_zone, seed_model_folder)
                # Process Model
                process_model(old_osm_path, new_osm_path, efficiency_level)
                shutil.move(old_epw_path, new_epw_path)
                # Prepare OSW
                v_epw_paths =  [f for f in os.listdir(sub_epws_path) if f.endswith('.epw')]
                v_osw_paths += prepare_all_osws(new_osm_path, v_epw_paths, out_osw_dir, measures_dir, n_runs)            
    f = open(os.join(root_directory, f"job_efficiency_level_{hash_eff_level[efficiency_level]}.txt"), mode = 'w')
    f.write(v_osw_paths)
    f.close
    return v_osw_paths

In [ ]:
def process_model(old_osm_path, new_osm_path, efficiency_level = 2):
    osm_dir = os.path.dirname(new_osm_path)
    Path(osm_dir).mkdir(parents = True, exist_ok = True)
    # Change Simulation Run Period to match Weather Data
    model = load_osm(old_osm_path)
    model.getSimulationControl.setRunSimulationforSizingPeriods(False)
    model.getSimulationControl.setRunSimulationforWeatherFileRunPeriods(True)
    # Enable CO2 Simulations
    model.getZoneAirContaminantBalance.setCarbonDioxideConcentration(True)
    # Change the VAV Control Logic to Dual Maximum
    vav_reheats = model.getAirTerminalSingleDuctVAVReheas()
    for vav_reheat in vav_reheats:
        vav_reheat.setDamperHeatingAction('ReverseWithLimits')
    # Change the Efficiency Level 
    model = adjust_efficiency_level(model, efficiency_level)
    # Save processed model
    model.save(new_osm_path, True)    

In [ ]:
def adjust_efficiency_level(model, level = 2):
    if level == 2:
        print(f"Keep the default Efficiency Level")
        return model
    elif level == 1:
        print(f"Adjusting to Low Efficiency Level")
        factor = 1.25
    elif level == 3:
        print(f"Adjusting to High Efficiency Level")
        factor = 0.75
    
    # Adjusting Lighting
    v_light_defs = model.getLightsDefinitions()
    for light_def in v_light_defs:
        old_lpd = light_def.wattsperSpaceFloorArea.to_f
        light_def.setWattsperSpaceFloorArea(old_lpd * factor)
    
    # Adjusting MELs
    v_equip_defs = model.getElectricEquipmentDefinitions
    for equip_def in v_equip_defs:
        if equip_def.designLevelCalculationMethod() == "Watts/Area":
            equip_def.setWattsperSpaceFloorArea(float(equip_def.wattsperSpaceFloorArea) * factor)
        elif equip_def.designLevelCalculationMethod() == "EquipmentLevel":
            equipp_def.setDesignLevel(float(equip_def.designLevel) * factor)
    
    # Wall Insulation
    v_opaque_materials = model.getStandardOpaqueMaterials()
    for opaque_material in v_opaque_materials:
        opaque_materials.setThermalConductivity(float(opaque_material.thermalConductivity) * factor)
    
    # Windows
    v_glazing_materials = model.getGlazings()
    for glazing_material in v_glazing_materials:
        glazing_material.setThickness(float(glazing_material.thickness())/factor)
    
    # Cooling Plant
    v_cooling_coils = model.getCoilCoolingDXTwoSpeeds()
    for cooling_coil in v_cooling_coils:
        cooling_coil.setRatedLowSpeedCOP(float(cooling_coil.ratedLowSpeedCOP)/factor)
        cooling_coil.setRatedHighSpeedCOP(float(cooling_coil.ratedHighSpeedCOP)/factor)
    
    # Heating Plant
    v_heating_coils = model.getCoilHeatingGass()
    for heating_coil in v_heating_coils:
        heating_coil.setGasBurnerEfficiency(min(0.95, float(heating_coil.gasBurnerEfficiency)/factor))
    
    # Reheating Coils
    v_reheating_coils = model.getCoilHeatingElectrics()
    for reheating_coil in v_reheating_coils:
        reheating_coil.setEfficiency(min(1, float(reheating_coil.efficiency)/factor))
    
    # Water Heaters
    v_water_heaters = model.getWaterHeaterMixeds
    for water_heater in v_water_heaters:
        water_heater.setHeaterThermalEfficiency(min(0.95, float(water_heater.heaterThermalEfficiency)/factor))
    
    # Fans
    v_fans = model.getFanVariableVolumnes()
    for fan in v_fans:
        fan.setFanTotalEfficiency(min(0.8, float(fan.fanTotalEfficiency)/factor))
        fan.setMotorEfficiency(min(0.95, float(fan.motorEfficiency)/factor))
    
    # Pumps
    v_pumps = model.getPumpConstantSpeeds()
    for pump in v_pumps:
        pump.setMotorEfficiency(min(0.6, float(pump.motorEfficiency)/factor))
    
    return model

In [ ]:
    def prepare_single_osw(seed_osm_path, epw_path, measures_dir, osw_path):
        Path(osw_path).mkdir(parents = True, exist_ok = True)
